L'objectif de cette partie est de faire une inpsection des données pour gérér les valeurs manquantes dans la bases construites avec les donnnées consommation + données météo + données rayonnement solaire


In [19]:
import pandas as pd
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import numpy as np



import os
import requests
import pandas as pd
import numpy as np
import datetime
import time
import sys
import gzip


In [20]:
from sklearn.base import BaseEstimator, TransformerMixin

# Une classe pour supprimer des colonnes dans la proportion des valeurs NAN  > 50
class ColumnDrop(BaseEstimator,TransformerMixin):
    # BaseEstimator contient les méthodes get_params et set_params.
    # TransformerMixin contient la méthode fit_transform.
    def __init__(self, seuil_for_drop):
        self.seuil_for_drop = seuil_for_drop
    def fit(self, X, y=None):
         # Ne fait rien
        return self
    def transform(self, X):
        # Supression des colonnes dont le pourcentage > seuil_for_drop 
        for col in X.columns:
            if (100*X[col].isna().sum()/len(X) > self.seuil_for_drop):
                X.drop(columns=[col], inplace=True)
        return X

In [21]:
# Une classe pour supprimer des lignes qui contiennent des valeurs NAN  pour une colonne donnée

class RowDrop(BaseEstimator,TransformerMixin):
    # BaseEstimator contient les méthodes get_params et set_params.
    # TransformerMixin contient la méthode fit_transform.
    def __init__(self, col_for_drop):
        # col_for_drop la colonne qu'on utilise pour supprimer les lignes contenant des nan
        self.col_for_drop = col_for_drop

    def fit(self, X, y=None):
         # Ne fait rien
        return self
    def transform(self, X):
        # # Supression des lignes dont la colonne Profil est nan
        rowindex_to_drop = X.loc[X[self.col_for_drop].isna()].index
        X.drop(index=rowindex_to_drop, inplace=True)
        return X

In [22]:
class GestionPas30Minutes(BaseEstimator,TransformerMixin):
    # BaseEstimator contient les méthodes get_params et set_params.
    # TransformerMixin contient la méthode fit_transform.
    def __init__(self):
        return None
    def fit(self, X, y=None):
         # Ne fait rien
        return self
    def transform(self, X):
        # Pour remplacer par la valeur la plus proche dans le temps, on ordonne par année, moi, jour et heure
        X.sort_values(['Profil', 'Plage de puissance souscrite','year', 'month', 'day', 'h'], ascending=[True, True,True,True, True, True], inplace=True)
        X['mn'] = X['mn'] + [0 if i % 2 == 0 else 30 for i in range(len(X))]
        return X

In [23]:
class GestionValeurManquante(BaseEstimator,TransformerMixin):
    # BaseEstimator contient les méthodes get_params et set_params.
    # TransformerMixin contient la méthode fit_transform.
    def __init__(self, column_name,mask, methode ="ffill"):
        self.column_name = column_name
        self.methode = methode
        self.mask = mask
    def fit(self, X, y=None):
         # Ne fait rien
        return self
    def transform(self, X):
        # Pour remplacer par la valeur la plus proche dans le temps, on ordonne par année, moi, jour et heure
        X.sort_values(['Profil', 'Plage de puissance souscrite','year', 'month', 'day', 'h'], ascending=[True, True,True,True, True, True], inplace=True)
        rolling_mean  = X.loc[self.mask, self.column_name].dropna().mean()
        X.loc[self.mask, self.column_name] = X.loc[self.mask, self.column_name].fillna(rolling_mean)
        return X

In [ ]:
# lecture des csv contenant les bases de données conso+meteo-rayonnement


folder_projet_BD = r"D:\MesDocuments\Formation\DataScientist_PSL\Projet\BD"
folder_BD_conso_meteo = os.path.join(folder_projet_BD, 'conso-inf36-meteo-rayonnement-region')

filename = 'departements-region.csv'
file = os.path.join(folder_projet_BD, filename)
df_depa_region = pd.read_csv(file)
df_depa_region['region_name'] = df_depa_region['region_name'].apply(lambda s : s.replace(" ", '-'))
Liste_region = df_depa_region['region_name'].unique().tolist()

for elem in ['Guadeloupe', 'Martinique', 'Guyane','La-Réunion', 'Mayotte', 'Corse']:
    Liste_region.remove(elem)

template_start_in= 'conso-inf36-meteo-rayonnement-'
template_end_in='-' + str(2023) + '-' + str(2024) + '.csv'



folder_BD_meteo_rayonnement_out = os.path.join(folder_projet_BD, 'conso-inf36-meteo-rayonnement-region-propre')
if not os.path.isdir(folder_BD_meteo_rayonnement_out):
    os.mkdir(folder_BD_meteo_rayonnement_out)


def nettoyage_BD(reg):
    filename = f"{template_start_in}{reg}{template_end_in}" 
    file = os.path.join(folder_BD_conso_meteo, filename)
    # lecture de la BD meteo de la région 
    df_fusion= pd.read_csv(file)


    # Gestion des minutes pour avoir le pas 1/2 h
    gestionminute = GestionPas30Minutes()
    gestionminute.fit_transform(df_fusion)

    # Nettoyage 
    # Suppresion des colonnes dont la proportion des nan dépasse 50%
    columndrop = ColumnDrop(50)
    columndrop.fit_transform(df_fusion)

    # Suppresion des ligne dont la colonne Profil contient des nan / profil inconnu !
    rowtodrop = RowDrop('Profil')
    rowtodrop.fit_transform(df_fusion)

    # Remplacement des Nan par des valeurs proches dans le temps selon profil et puissance inscrite
    for profile in df_fusion['Profil'].unique():
        for valeur_puissance in df_fusion.loc[df_fusion['Profil']==profile]['Plage de puissance souscrite'].unique():
            mask = (df_fusion['Profil'] == profile) & (df_fusion['Plage de puissance souscrite'] ==valeur_puissance)
            for col in df_fusion.columns:
                if (df_fusion.loc[mask,col].isna().sum() > 0 ):
                    if(100*df_fusion.loc[mask,col].isna().sum() / len(df_fusion.loc[mask,col]) > 80): # proportion_nan> 50%
                        rowindex_to_drop = df_fusion.loc[mask,col].isna().index
                        df_fusion.drop(index=rowindex_to_drop, inplace=True)
                        #print('>80%')
                    else :
                        gestionvaleurmanquante= GestionValeurManquante(column_name=col, mask = mask, methode ='ffill')
                        gestionvaleurmanquante.fit_transform(df_fusion)


    
    # Sauvegarde
    
    file_sorie = os.path.join(folder_BD_meteo_rayonnement_out, filename)
    df_fusion.to_csv(file_sorie, index=False)
    print('fin traitement', reg)


for reg in Liste_region:
    # Lecture de la base 
   nettoyage_BD(reg)




fin traitement Auvergne-Rhône-Alpes
fin traitement Hauts-de-France
fin traitement Provence-Alpes-Côte-d'Azur
fin traitement Grand-Est
fin traitement Occitanie
fin traitement Normandie
fin traitement Nouvelle-Aquitaine
fin traitement Centre-Val-de-Loire
fin traitement Bourgogne-Franche-Comté
fin traitement Bretagne
fin traitement Pays-de-la-Loire
fin traitement Île-de-France


In [ ]:
# lecture des csv contenant les bases de données conso+meteo-rayonnement


folder_projet_BD = r"D:\MesDocuments\Formation\DataScientist_PSL\Projet\BD"
folder_BD_meteo_rayonnement_out = os.path.join(folder_projet_BD, 'conso-inf36-meteo-rayonnement-region-propre')

filename = 'departements-region.csv'
file = os.path.join(folder_projet_BD, filename)
df_depa_region = pd.read_csv(file)
df_depa_region['region_name'] = df_depa_region['region_name'].apply(lambda s : s.replace(" ", '-'))
Liste_region = df_depa_region['region_name'].unique().tolist()

for elem in ['Guadeloupe', 'Martinique', 'Guyane','La-Réunion', 'Mayotte', 'Corse']:
    Liste_region.remove(elem)

template_start_in= 'conso-inf36-meteo-rayonnement-'
template_end_in='-' + str(2023) + '-' + str(2024) + '.csv'




def nettoyage_heure_BD(reg):
    filename = f"{template_start_in}{reg}{template_end_in}" 
    file = os.path.join(folder_BD_meteo_rayonnement_out, filename)
    # lecture de la BD meteo de la région 
    df_fusion= pd.read_csv(file, low_memory=False)
    df_fusion.h =df_fusion.h - 1
    df_fusion.to_csv(file, index=False)
    print('fin traitement', reg)


for reg in Liste_region:
   print(reg)
   nettoyage_heure_BD(reg)
   
   



Auvergne-Rhône-Alpes
fin traitement Auvergne-Rhône-Alpes
Hauts-de-France
fin traitement Hauts-de-France
Provence-Alpes-Côte-d'Azur
fin traitement Provence-Alpes-Côte-d'Azur
Grand-Est
fin traitement Grand-Est
Occitanie
fin traitement Occitanie
Normandie
fin traitement Normandie
Nouvelle-Aquitaine
fin traitement Nouvelle-Aquitaine
Centre-Val-de-Loire
fin traitement Centre-Val-de-Loire
Bourgogne-Franche-Comté
fin traitement Bourgogne-Franche-Comté
Bretagne
fin traitement Bretagne
Pays-de-la-Loire
fin traitement Pays-de-la-Loire
Île-de-France
fin traitement Île-de-France


In [ ]:
# lecture des csv contenant les bases de données conso+meteo-rayonnement
# Suppresion des doublons


folder_projet_BD = r"D:\MesDocuments\Formation\DataScientist_PSL\Projet\BD"
folder_BD_meteo_rayonnement_out = os.path.join(folder_projet_BD, 'conso-inf36-meteo-rayonnement-region-propre')

filename = 'departements-region.csv'
file = os.path.join(folder_projet_BD, filename)
df_depa_region = pd.read_csv(file)
df_depa_region['region_name'] = df_depa_region['region_name'].apply(lambda s : s.replace(" ", '-'))
Liste_region = df_depa_region['region_name'].unique().tolist()

for elem in ['Guadeloupe', 'Martinique', 'Guyane','La-Réunion', 'Mayotte', 'Corse']:
    Liste_region.remove(elem)

template_start_in= 'conso-inf36-meteo-rayonnement-'
template_end_in='-' + str(2023) + '-' + str(2024) + '.csv'




def suppression_doublon_BD(reg):
    filename = f"{template_start_in}{reg}{template_end_in}" 
    file = os.path.join(folder_BD_meteo_rayonnement_out, filename)
    # lecture de la BD meteo de la région 
    df_fusion= pd.read_csv(file, low_memory=False)
    df_fusion.drop_duplicates(inplace=True)
    df_fusion.to_csv(file, index=False)
    print('fin traitement', reg)


for reg in Liste_region:
    # Lecture de la base 
   suppression_doublon_BD(reg)

In [ ]:

folder_projet_BD = r"D:\MesDocuments\Formation\DataScientist_PSL\Projet\BD"
folder_BD_propre= os.path.join(folder_projet_BD, 'conso-inf36-meteo-rayonnement-region-propre')
liste_file = os.listdir(folder_BD_propre)
reg = 0
file = os.path.join(folder_BD_propre,liste_file[reg]) #'conso-inf36-Auvergne-Rhône-Alpes-2023-2024.csv') #
df = pd.read_csv(file, low_memory=False)

# isna
100*df.isna().sum()/len(df)

Région                                          0.0
Code région                                     0.0
Profil                                          0.0
Plage de puissance souscrite                    0.0
Nb points soutirage                             0.0
Total énergie soutirée (Wh)                     0.0
Courbe Moyenne n°1 (Wh)                         0.0
Indice représentativité Courbe n°1 (%)          0.0
Courbe Moyenne n°2 (Wh)                         0.0
Indice représentativité Courbe n°2 (%)          0.0
Courbe Moyenne n°1 + n°2 (Wh)                   0.0
Indice représentativité Courbe n°1 + n°2 (%)    0.0
Jour max du mois (0/1)                          0.0
Semaine max du mois (0/1)                       0.0
AAAAMMJJHH                                      0.0
T_moyenne                                       0.0
T_STD                                           0.0
T_min                                           0.0
T_q25                                           0.0
T_q50       